<a href="https://colab.research.google.com/github/Binitaaaaaa/movie_recommendation_system/blob/main/Collaborative_filteringusing_matrixfact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the datasets
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

In [2]:
ratings_df.shape

(100836, 4)

In [3]:
movies_df.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [4]:
ratings_df.head(1)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703


In [5]:
movies = pd.merge(ratings_df, movies_df, on='movieId')


In [6]:
movies.head(1)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [7]:
# Remove duplicates if any
movies = movies.drop_duplicates()


In [8]:
# Create the User-Item Rating Matrix (pivot table)
user_item_matrix = movies.pivot(index='userId', columns='movieId', values='rating')

# Display the shape and first few entries of the matrix
print(user_item_matrix.shape)
print(user_item_matrix.head())


(610, 9724)
movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     NaN     4.0     NaN     NaN     4.0     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           4.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  9       10      ...  193565  193567  193571  193573  193579  193581  \
userId                   ...                                                   
1           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN  ...     NaN     NaN     NaN     NaN  

In [9]:
# Calculate the global average rating
global_avg_rating = movies['rating'].mean()

# Fill NaN values with the global average rating
user_item_matrix_filled = user_item_matrix.fillna(global_avg_rating)


In [10]:
user_item_matrix_filled.head(1)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,3.501557,4.0,3.501557,3.501557,4.0,3.501557,3.501557,3.501557,3.501557,...,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557


In [ ]:
# from sklearn.decomposition import TruncatedSVD
# import numpy as np

# # Perform SVD (Singular Value Decomposition)
# svd = TruncatedSVD(n_components=50)  # Number of latent features (components)
# user_item_matrix_svd = svd.fit_transform(user_item_matrix_filled)

# # The result is a matrix with reduced dimensions
# print(user_item_matrix_svd.shape)


In [11]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357285 sha256=22092d50528b7e71026609a863185a9d14b8a63c82da17f865242c2cdd7064ca
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [12]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Prepare data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(movies[['userId', 'movieId', 'rating']], reader)

# Split data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Initialize SVD model
svd = SVD()

# Train the model
svd.fit(trainset)

# Predict ratings on the testset
predictions = svd.test(testset)

# Evaluate the model using RMSE
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')


RMSE: 0.8751
RMSE: 0.8750632858406394


In [13]:
# Predict the rating for a specific user-item pair
user_id = 1  # Example user
movie_id = 50  # Example movie

predicted_rating = svd.predict(user_id, movie_id).est
print(f"Predicted rating for User {user_id} on Movie {movie_id}: {predicted_rating}")


Predicted rating for User 1 on Movie 50: 5


In [16]:
import heapq

def recommend_movies(user_id, top_n=10):
    """Recommends top N movies for a specific user.

    Args:
        user_id (int): The ID of the user.
        top_n (int, optional): The number of movies to recommend. Defaults to 10.

    Returns:
        list: A list of movie titles.
    """

    # Get all movie IDs and create a dictionary for movieId -> title lookup
    movie_id_to_title = dict(zip(movies['movieId'], movies['title']))

    # Get all unique movie IDs
    all_movie_ids = movies['movieId'].unique()

    # Get the movies that the user has already rated
    rated_movies = set(movies[movies['userId'] == user_id]['movieId'])

    # Get the unrated movies
    unrated_movies = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movies]

    # Predict ratings for unrated movies
    predictions = [svd.predict(user_id, movie_id) for movie_id in unrated_movies]

    # Sort movies by predicted rating (highest first)
    recommended_movies = heapq.nlargest(top_n, predictions, key=lambda x: x.est)

    # Get movie titles from the top N predictions using the dictionary
    movie_titles = [movie_id_to_title[rec.iid] for rec in recommended_movies]

    return movie_titles

# Example usage:
user_id = 1  # Example user ID
recommended_movies = recommend_movies(user_id, top_n=10)
print(f"Top 10 movie recommendations for User {user_id}: {recommended_movies}")

Top 10 movie recommendations for User 1: ['Shawshank Redemption, The (1994)', 'Departed, The (2006)', 'Dark Knight, The (2008)', 'Inglourious Basterds (2009)', 'My Fair Lady (1964)', 'Philadelphia Story, The (1940)', 'Rear Window (1954)', 'Notorious (1946)', 'Brazil (1985)', '12 Angry Men (1957)']


In [17]:
user_id_input = input("Enter your user ID: ")
try:
    user_id = int(user_id_input)  # Ensure the user ID is an integer
    recommended_movies = recommend_movies(user_id, top_n=10)
    print(f"Top 10 movie recommendations for User {user_id}: {recommended_movies}")
except ValueError:
    print("Invalid input. Please enter a valid integer for user ID.")

Enter your user ID: 2
Top 10 movie recommendations for User 2: ['Great Escape, The (1963)', 'Philadelphia Story, The (1940)', "Schindler's List (1993)", 'Monty Python and the Holy Grail (1975)', '12 Angry Men (1957)', 'Hustler, The (1961)', 'Lawrence of Arabia (1962)', 'North by Northwest (1959)', 'Godfather, The (1972)', 'Chinatown (1974)']
